In [2]:
import pandas as pd
import json

In [35]:
def load_Data(user, filename):
   data = json.load(open('dir.json'))
   dir = data[user] + filename
#    data = pd.read_csv(dir, header=2)
   return dir

In [37]:
data = pd.read_csv(load_Data('Yash', 'AG01-01.csv'), header=0)
data.head()

,Unnamed: 0,mean.vm,sd.vm,mean.ang,sd.ang,p625,dfreq,ratio.df,start.time
0,1,1.008720,0.005388,14.405906,0.178485,0.073094,6.153846,0.079059,10/2/2017 10:15
1,2,1.008625,0.006601,14.444647,0.234531,0.074642,4.102564,0.174183,10/2/2017 10:15
2,3,1.018935,0.023567,12.862005,2.996361,0.309095,1.025641,0.158892,10/2/2017 10:15
3,4,1.019761,0.066683,-1.952166,12.679718,0.114479,2.051282,0.101853,10/2/2017 10:15
4,5,1.043682,0.056193,-16.001678,5.775647,0.217691,1.025641,0.149609,10/2/2017 10:15
